<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Parameter-settings" data-toc-modified-id="Parameter-settings-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Parameter settings</a></span></li><li><span><a href="#PFLOTRAN-preparation-&amp;-spin-up" data-toc-modified-id="PFLOTRAN-preparation-&amp;-spin-up-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>PFLOTRAN preparation &amp; spin-up</a></span><ul class="toc-item"><li><span><a href="#Generate-PFLOTRAN.in-and-parameter.h5" data-toc-modified-id="Generate-PFLOTRAN.in-and-parameter.h5-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Generate <code>PFLOTRAN.in</code> and <code>parameter.h5</code></a></span></li><li><span><a href="#Model-spin-up" data-toc-modified-id="Model-spin-up-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Model spin-up</a></span></li></ul></li><li><span><a href="#DART-files-preparation" data-toc-modified-id="DART-files-preparation-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>DART files preparation</a></span><ul class="toc-item"><li><span><a href="#Generate-the-templates-for-DART-generic-variable-quantity-files" data-toc-modified-id="Generate-the-templates-for-DART-generic-variable-quantity-files-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Generate the templates for DART generic variable quantity files</a></span></li><li><span><a href="#Preprocess-the-DART-generic-variable-quantity-files" data-toc-modified-id="Preprocess-the-DART-generic-variable-quantity-files-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span><code>Preprocess</code> the DART generic variable quantity files</a></span></li><li><span><a href="#Convert-observation-to-DART-observation-format" data-toc-modified-id="Convert-observation-to-DART-observation-format-3.3"><span class="toc-item-num">3.3&nbsp;&nbsp;</span>Convert observation to DART observation format</a></span></li><li><span><a href="#(TODO)-Generate-prior.nc-from-the-model-spin-up" data-toc-modified-id="(TODO)-Generate-prior.nc-from-the-model-spin-up-3.4"><span class="toc-item-num">3.4&nbsp;&nbsp;</span>(TODO) Generate <code>prior.nc</code> from the model spin-up</a></span></li><li><span><a href="#(TODO)-Generate-input.nml" data-toc-modified-id="(TODO)-Generate-input.nml-3.5"><span class="toc-item-num">3.5&nbsp;&nbsp;</span>(TODO) Generate <code>input.nml</code></a></span></li><li><span><a href="#Run-check_model_mod" data-toc-modified-id="Run-check_model_mod-3.6"><span class="toc-item-num">3.6&nbsp;&nbsp;</span>Run <code>check_model_mod</code></a></span></li></ul></li><li><span><a href="#(TODO)-Run-DART-and-PFLOTRAN" data-toc-modified-id="(TODO)-Run-DART-and-PFLOTRAN-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>(TODO) Run DART and PFLOTRAN</a></span></li></ul></div>

# Parameter settings

In [70]:
# Directories
obs_kind_dir    = '../obs_kind/'
obs_type_dir    = '../obs_type/'
obs_convert_dir = '../obs_converter/'
utils_dir       = '../utils/'
work_dir        = '../work/'
pflotran_in_dir = '../pflotran_input/'
pflotran_out_dir= '../pflotran_output/'
dart_data_dir   = '../dart_inout/'
# data_dir        = '../data_files/'

# DART file names
def_obs_kind = obs_kind_dir+'DEFAULT_obs_kind_mod.f90'
obs_type     = obs_type_dir+'obs_def_pflotran_mod.f90'
input_nml    = utils_dir+'input.nml'

# PFLOTRAN file names
pflotran_sh   = utils_dir+'pflotran.sh'
pflotran_exe  = '/Users/jian449/Codes/pflotran/src/pflotran/pflotran'
pflotran_in   = pflotran_in_dir+'pflotran.in'
pflotran_para = pflotran_in_dir+'parameter.h5'
pflotran_out  = pflotran_out_dir+'pflotran.out'

# Data file names, including observations and model input/output
obs_original = pflotran_in+'temperature.csv'
obs_nc       = pflotran_in+'obs_pflotran.nc'
obs_dart     = dart_data_dir+'obs_seq_pflotran.out'
prior_nc     = dart_data_dir+'prior_template.nc'

# Some shell scripts
compile_convert_nc      = obs_convert_dir+'dart_seq_convert.csh'
compile_model_check_mod = work_dir+'check_model_mod.csh'
run_filter              = work_dir+'run_filter.csh'

# MPI settings
mpi_exe = '/usr/local/bin/mpirun'
ncore   = 2

# Utility file names
csv_to_nc          = utils_dir+'csv2nc.py'
convert_nc         = obs_convert_dir+'convert_nc'
to_dartqty         = utils_dir+'list2dartqty.py'
obs_type_qty_ind   = utils_dir+'obs_type_qty_ind.txt'
prep_pflotran_in   = utils_dir+'prepare_pflotran_inpara.py'
prep_obs_nc        = utils_dir+'prepare_obs_nc.py'
prep_prior_nc      = utils_dir+'prepare_prior_nc.py'
prep_inputnml      = utils_dir+'prepare_inputnml.py'

In [68]:
# Data assimilation configurations
# More need to be added...
# And later on, these DA setting can be saved in a txt or pickel file for further loading
obs_timestep = 300.0  # second
obs_error    = 0.1    # observation error
spinup       = 1      # whether model spinup
nens         = 30     # number of ensembles

# Specify PFLOTRAN variables used as observation and state vector/parameters in DART
obs_var_set = ['TEMPERATURE']
para_set    = ['FLUX','POROSITY','THERMALCONDUCTIVITY']
# state_set   = ['LIQUID_SATURATION','LIQUID_PRESSURE', 'TEST_VARIABLE', 'TEST_VARIABLEAAA'
pflotran_parastate_set = obs_var_set + para_set

# PFLOTRAN preparation & spin-up
**Here, we use Kewei's 1D thermal model as an example for generating PFLOTRAN input card and parameter.h5.**

In this section, the following procedures are conducted for PFLOTRAN:
- generate ```PFLOTRAN.in``` (for 1D thermal model, check Kewei's code)
- generate the parameter files in HDF 5, ```parameter.h5``` (for 1D thermal model, check Kewei's code)
- conduct model spin-up and generate PFLOTRAN output in HDF 5 format, ```PFLOTRAN.h5```

## Generate ```PFLOTRAN.in``` and ```parameter.h5```
- Run: ```prepare_pflotran_inpara.py```
- Code input arguments:
    - filename for ```pflotran.in```
    - filename for ```parameter.h5```
    - data assimilation settings (**to be revised later on**): observation timestep, observation error, number of ensemble, whether it is spinup

In [86]:
%%script bash -s "$prep_pflotran_in" "$pflotran_in" "$pflotran_para" "$obs_timestep" "$obs_error" "$nens" "$spinup"
python $1 $2 $3 $4 $5 $6 $7

Finished generating the input card for PFLOTRAN...
Finished generating the DBASE for PFLOTRAN...


## Model spin-up
Take in the ```pflotran.in``` and ```parameter.h5``` files and conduct the model spin-up by running ```pflotran.sh``` file. The ```pflotran.sh``` is a simple shell script executing ensemble simulation of PFLOTRAN by using MPI.

In [88]:
%%script bash -s "$pflotran_sh" "$pflotran_exe" "$pflotran_in" "$pflotran_out_dir" "$nens" "$mpi_exe" "$ncore"
echo $3
$1 $2 $3 $4 $5 $6 $7

../pflotran_input/pflotran.in
../pflotran_input/pflotran.in
 here
          30
 here
          30
Finished running PFLOTRAN...


HDF5-DIAG: Error detected in HDF5 (1.8.18) MPI-process 0:
  #000: H5D.c line 165 in H5Dcreate2(): not a location ID
    major: Invalid arguments to routine
    minor: Inappropriate type
  #001: H5Gloc.c line 253 in H5G_loc(): invalid object ID
    major: Invalid arguments to routine
    minor: Bad value
HDF5-DIAG: Error detected in HDF5 (1.8.18) MPI-process 0:
  #000: H5Dio.c line 228 in H5Dwrite(): not a dataset
    major: Invalid arguments to routine
    minor: Inappropriate type
HDF5-DIAG: Error detected in HDF5 (1.8.18) MPI-process 0:
  #000: H5D.c line 415 in H5Dclose(): not a dataset
    major: Invalid arguments to routine
    minor: Inappropriate type
HDF5-DIAG: Error detected in HDF5 (1.8.18) MPI-process 0:
  #000: H5D.c line 165 in H5Dcreate2(): not a location ID
    major: Invalid arguments to routine
    minor: Inappropriate type
  #001: H5Gloc.c line 253 in H5G_loc(): invalid object ID
    major: Invalid arguments to routine
    minor: Bad value
HDF5-DIAG: Error detected in

# DART files preparation
In this section, the following procedures are conducted for DART:
- generate the template for DART generic variable quantity files (i.e., ```DEFAULT_obs_kind_mod.F90``` and ```obs_def_pflotran_mod.f90```);
- run ```preprocess``` to generate the rest of DART generic variable quantity files;
- convert the observation file to DART observation format;
- generate ```prior.nc``` at the first time step;
- generate ```input.nml```;
- conduct ```check_model_mod```.

## Generate the templates for DART generic variable quantity files
- Run: ```list2dartqty.py```
- Code input arguments:
    - filename for ```DEFAULT_obs_kind_mod.F90```
    - filename for ```obs_def_pflotran_mod.F90```
    - a list of variables required to be assimilated

In [ ]:
%%script bash -s "$to_dartqty" "$obs_type" "$def_obs_kind" "$pflotran_parastate_set"
python $1 $2 $3 $4

## ```Preprocess``` the DART generic variable quantity files
- Run shell script: ```dart_seq_convert.csh```
- This shell script 
    - preprocesses the DART generic variable quantity files prepared by the previous section
    - generates an executable file for converting observation file in NetCDF format to DART format used by the next section

In [ ]:
%%script bash -s "$obs_convert_dir" "$compile_convert_nc"
cd $1
csh $2

## Convert observation to DART observation format
- Run: ```prepare_obs_nc.py``` to convert the raw csv temperature observations to NetCDF file
- Run: ```convert_nc``` to convert the observation file in NetCDF to DART format

In [ ]:
# Convert the observation to NetCDF format
%%script bash -s "$prep_obs_nc" "$obs_nc"
python $1 $2

In [ ]:
# Convert the observation to DART format
%%script bash -s "$obs_convert_dir" "$convert_nc"
cd $1
$2

## (TODO) Generate ```prior.nc``` from the model spin-up
- Run: ```prepare_prior_nc.py```
- Code input arguments:
    - filename ```prior_template.nc```
    - pflotran parameter HDF file ```parameter.h5```
    - a list of variables to be assimilated

In [ ]:
%%script bash -s "$prep_prior_nc" "$prior_nc" "$pflotran_para" "$pflotran_parastate_set"
python $1 $2 $3 $4

## (TODO) Generate ```input.nml```
- Run: ```prepare_inputnml.py```
- Code input arguments:
    - filename ```input_nml```
    - filename ```prior_template.nc```
    - a list of variables to be assimilated

In [ ]:
%%script bash -s  "$prep_inputnml" "$input_nml" "$prior_nc" "$pflotran_parastate_set"
python $1 $2 $3 $4

## Run ```check_model_mod```
- Run shell script: ```dart_seq_convert.csh```
- This shell script 
    - preprocesses the DART generic variable quantity files prepared by the previous section
    - generates an executable file for converting observation file in NetCDF format to DART format used by the next section

In [ ]:
%%script bash -s "$work_dir" "$compile_model_check_mod"
cd $1
csh $2

# (TODO) Run DART and PFLOTRAN
In this section, run the shell script to couple DART and PFLOTRAN

In [ ]:
%%script bash -s "$work_dir" "$run_filter"
cd $1
csh $2